# Summary of Access to Opportunity Results
This workbook summarizes and provides visualizations of access to opportunities for a project based on the information and detail supplied in a given project's configuration file.

This workbook is intended as a development tool to help design visualizations and to provide a quick look at results.

In [1]:
import pandas as pd
import altair as alt
import yaml
from datetime import datetime, timedelta
import os
os.environ['USE_PYGEOS'] = '0'

# Read in configuration details
uid = "septa-redesign-weekend"
with open(f"../cache/{uid}/config.yml") as settings_file:
    config = yaml.safe_load(settings_file)

s0_name = config['scenarios'][0]['name']
s1_name = config['scenarios'][1]['name']
s0_start = datetime.strptime(config['scenarios'][0]['start_datetime'], '%Y-%m-%d %H:%M').strftime("%b %-d, %Y from %-I:%M%p")
s0_end = (datetime.strptime(config['scenarios'][0]['start_datetime'], '%Y-%m-%d %H:%M') + timedelta(minutes=config["scenarios"][0]["duration"])).strftime("%-I:%M%p")
s1_start = datetime.strptime(config['scenarios'][1]['start_datetime'], '%Y-%m-%d %H:%M').strftime("%b %-d, %Y from %-I:%M%p")
s1_end = (datetime.strptime(config['scenarios'][1]['start_datetime'], '%Y-%m-%d %H:%M') + timedelta(minutes=config["scenarios"][0]["duration"])).strftime("%-I:%M%p")

# Read in summary data
summary = pd.read_csv(f"../cache/{uid}/summary.csv")
summary.head()

,metric,B03002_001E,B03002_003E,B03002_004E,B03002_006E,B03002_012E,C17002_003E
0,C000_c30_0,43959.442284,41281.674927,44701.254519,62830.149391,39668.853331,45548.875065
1,C000_c45_0,136132.457045,95684.965254,184132.395786,149917.972563,168362.856621,184103.603569
2,hospitals_t1_0,53.641093,65.862504,39.307057,52.838139,42.086931,41.166650
3,rental_units_c30_0,422.607430,330.878656,512.948806,534.903999,493.533229,526.241587
4,rental_units_c45_0,1454.282608,1045.819747,1918.310105,1641.741915,1800.557071,1933.844883


## Differences in Population Demographics Across A Single Variable
Here we provide the ability to chart differences in demographic group distributions for single destinations.

In [2]:
# Let's start by fetching only the data that isn't a comparison
absolutes = summary[summary['metric'].str[-3:] != "1-0"].copy()
# Next, we split our data into scenario columns
absolutes['scenario'] = absolutes["metric"].str[-1]
# to_plot['metric_name'] = 
absolutes['parameter'] = absolutes["metric"].str[:-2].str.split("_").str[-1]
absolutes['opportunity'] = absolutes["metric"].str[:-2].str.split("_").str[:-1]
absolutes['opportunity'] = absolutes['opportunity'].str.join("_")
absolutes = absolutes.drop(columns=['metric'])
absolutes = absolutes.melt(id_vars=["opportunity", "parameter", "scenario"])
opp_dict = {}
for k in config["opportunities"].keys():
    opp_dict[k] = config["opportunities"][k]["name"]

scen_dict = {}
for idx, s in enumerate(config["scenarios"]):
    scen_dict[str(idx)] = s['name']

absolutes["scenario"] = absolutes["scenario"].map(scen_dict)
absolutes['variable'] = absolutes['variable'].map(config["demographics"])
absolutes['opportunity'] = absolutes['opportunity'].map(opp_dict)
absolutes.head()

,opportunity,parameter,scenario,variable,value
0,Total Employment,c30,Business as Usual,Everyone,43959.442284
1,Total Employment,c45,Business as Usual,Everyone,136132.457045
2,Hospitals,t1,Business as Usual,Everyone,53.641093
3,Rental Units,c30,Business as Usual,Everyone,422.607430
4,Rental Units,c45,Business as Usual,Everyone,1454.282608


In [3]:
opportunity = "Total Employment"
parameter = "c45"

to_plot = absolutes.query(f"opportunity == '{opportunity}' and parameter == '{parameter}'").copy()

bars = alt.Chart().mark_bar(color="lightgrey", height=3).encode(
    alt.X("value:Q", title="Total Jobs Accessible in 45 Minutes"),
    alt.Y("scenario:N", axis=alt.Axis(labels=False, title=None))
)

circles = alt.Chart(width=600).mark_circle(size=150, opacity=1).encode(
    alt.X("value:Q"),
    alt.Y("scenario:N"),
    alt.Color("scenario:N", title="Scenario")
)


alt.layer(bars, circles, data=to_plot).facet(
    row=alt.Row("variable:N", title=None, header=alt.Header(labelAngle=0, labelAlign="left", labelFontSize=12))
).properties(
    title={
        "text": f"{config['project']} - Access to Jobs",
        "subtitle": [f"{s0_name}: {s0_start} to {s0_end}", f"{s1_name}: {s1_start} to {s1_end}"]
    },
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    fontSize=18,
    anchor="middle"
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure_legend(
    titleFontSize=14,
    labelFontSize=12,
    orient="top"
)

alt.FacetChart(...)

In [4]:
# Let's start by fetching only the data that isn't a comparison
deltas = summary[summary['metric'].str[-3:] == "1-0"].copy()

# to_plot['metric_name'] = 
deltas['parameter'] = deltas["metric"].str[:-2].str.split("_").str[-2]
deltas['opportunity'] = deltas["metric"].str[:-2].str.split("_").str[:-2].str.join("_")
# deltas['opportunity'] = deltas['opportunity'].str.join("_")
deltas = deltas.drop(columns=['metric'])
deltas = deltas.melt(id_vars=["opportunity", "parameter"])

# opp_dict = {}
# for k in config["opportunities"].keys():
#     opp_dict[k] = config["opportunities"][k]["name"]

deltas['variable'] = deltas['variable'].map(config["demographics"])
deltas['opportunity'] = deltas['opportunity'].map(opp_dict)
deltas.head()

,opportunity,parameter,variable,value
0,Total Employment,c30,Everyone,2343.885305
1,Total Employment,c45,Everyone,5254.730364
2,Hospitals,t1,Everyone,2.041504
3,Rental Units,c30,Everyone,51.088354
4,Rental Units,c45,Everyone,112.924494


In [5]:
to_plot = deltas.query("opportunity == 'Total Employment' and parameter == 'c45'").copy()

bars = alt.Chart(to_plot).mark_bar(color="lightgrey", height=3).encode(
    alt.X("value:Q", title="Change in Jobs Accessible in 45 Minutes"),
    alt.Y("variable:N", title=None)
)

circles = alt.Chart(to_plot).mark_circle(color="#823BA0", size=150, opacity=1).encode(
    alt.X("value:Q"),
    alt.Y("variable:N")
)

(bars+circles).properties(
    title={
        "text": f"{config['project']} - Change in Access to Jobs",
        "subtitle": [f"{s0_name}: {s0_start} to {s0_end}", f"{s1_name}: {s1_start} to {s1_end}"]
    },
    height=100, 
    width=600
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    fontSize=18,
    anchor="start"
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
)

alt.LayerChart(...)

## Summary in Small Multiples

In [6]:
to_plot = deltas.copy()
to_plot['combined_name'] = deltas['opportunity'] + " (" + deltas["parameter"] + ")"

bars = alt.Chart().mark_bar(color="lightgrey", height=3).encode(
    alt.X("value:Q", title="Change in Measure"),
    alt.Y("variable:N", title=None)
)

circles = alt.Chart().mark_circle(color="#559613", size=150, opacity=1).encode(
    alt.X("value:Q"),
    alt.Y("variable:N")
)

alt.layer(bars, circles, data=to_plot).facet(
    facet=alt.Facet("combined_name:N", title=None, header=alt.Header(labelFontWeight='bold', labelFontSize=12)),
    columns=2
).properties(
    title={
        "text": f"{config['project']} - Summary of Changes",
        "subtitle": [f"{s0_name}: {s0_start} to {s0_end}", f"{s1_name}: {s1_start} to {s1_end}"]
    },
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    fontSize=18,
    subtitleFontSize=14,
    anchor="middle"
).resolve_scale(x='independent')

alt.FacetChart(...)

In [7]:
absolutes.head()

,opportunity,parameter,scenario,variable,value
0,Total Employment,c30,Business as Usual,Everyone,43959.442284
1,Total Employment,c45,Business as Usual,Everyone,136132.457045
2,Hospitals,t1,Business as Usual,Everyone,53.641093
3,Rental Units,c30,Business as Usual,Everyone,422.607430
4,Rental Units,c45,Business as Usual,Everyone,1454.282608


In [8]:
to_plot = absolutes.copy()
to_plot['combined_name'] = absolutes['opportunity'] + " (" + absolutes["parameter"] + ")"
alt.Chart(to_plot).mark_point(size=100, fillOpacity=0).encode(
    alt.X("variable:N", title=None, sort=["Everyone"]),
    alt.Y("value:Q", title="Average Score"),
    alt.Color("scenario:N", title="Scenario"),
    alt.Facet("combined_name:N", title="", columns=3, header=alt.Header(labelFontSize=12, labelFontWeight="bold"))
).properties(
    width=300,
    height=200,
    title={
        "text": f"{config['project']} - Summary of Absolute Values",
        "subtitle": [f"{s0_name}: {s0_start} to {s0_end}", f"{s1_name}: {s1_start} to {s1_end}"]
    },
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    fontSize=18,
    subtitleFontSize=14,
    anchor="middle"
).configure_axisX(
    labelAngle=-30,
    labelFontSize=12
).resolve_scale(y='independent', x="independent")

alt.Chart(...)

## Maps!

In [9]:
import geopandas as gpd
bgs = gpd.read_file(f"../cache/{uid}/analysis_polygons.geojson", dtype={"GEOID":str})
compared = pd.read_csv(f"../cache/{uid}/compared.csv", dtype={"bg_id":str})
impact = pd.read_csv(f"../cache/{uid}/impact_area.csv", dtype={"bg_id":str})
# Join and match the compared file
bgs = bgs[bgs.GEOID.isin(impact.bg_id)]
bgs = pd.merge(bgs, compared, left_on="GEOID", right_on="bg_id")
bgs.describe()

,C000_c30_0,C000_c45_0,hospitals_t1_0,rental_units_c30_0,rental_units_c45_0,corridor_acres_c30_0,corridor_acres_c45_0,supermarkets_t1_0,supermarkets_t3_0,C000_c30_1,...,supermarkets_t3_1,C000_c30_1-0,C000_c45_1-0,hospitals_t1_1-0,rental_units_c30_1-0,rental_units_c45_1-0,corridor_acres_c30_1-0,corridor_acres_c45_1-0,supermarkets_t1_1-0,supermarkets_t3_1-0
count,2207.00000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,...,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000,2207.000000
mean,52061.36928,153049.432261,51.547802,491.273675,1662.331219,493.665310,1690.739082,24.783416,38.655641,54955.120072,...,38.737653,2893.750793,5138.421386,1.849570,61.713638,123.523788,35.864068,69.131164,0.157227,0.082012
std,95019.28654,177209.134891,41.603440,795.914326,2165.015962,752.210985,1919.954933,21.039169,23.381462,98984.629674,...,23.095265,14971.983157,20555.401508,12.376609,178.253109,318.548545,120.265693,225.489370,7.789922,9.268048
min,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,...,7.000000,-104034.000000,-251408.000000,-111.000000,-604.000000,-2234.000000,-389.790730,-1838.312290,-131.000000,-121.000000
25%,3391.50000,11664.000000,24.000000,0.000000,0.000000,0.000000,0.000000,13.000000,23.000000,3331.500000,...,23.000000,-214.000000,-1620.500000,-1.000000,0.000000,0.000000,0.000000,-6.938146,0.000000,-1.000000
50%,9137.00000,44601.000000,35.000000,53.000000,383.000000,188.286475,935.694122,21.000000,31.000000,9796.000000,...,31.000000,0.000000,776.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,30975.00000,347979.000000,69.000000,743.000000,3226.000000,602.217369,3081.300507,30.000000,48.000000,34543.000000,...,49.000000,1887.000000,9136.000000,2.000000,62.000000,191.000000,47.023374,146.827537,0.000000,2.000000
max,435462.00000,563019.000000,180.000000,5239.000000,7908.000000,4465.659135,7302.515301,180.000000,180.000000,439462.000000,...,180.000000,139907.000000,141251.000000,119.000000,1470.000000,2219.000000,1049.317410,1145.038064,124.000000,113.000000


In [10]:
alt.Chart(bgs).mark_geoshape(clip=True).encode(
    color=alt.Color('supermarkets_t1_1-0:Q', title="Added Travel Time", scale=alt.Scale(domain=[-10, 10], type="bin-ordinal", scheme="blueorange"))
).properties(
    width=800,
    height=500,
    title={
        "text": f"{config['project']} - Change in Access to Supermarkets",
        "subtitle": [
            f"{s0_name}: {s0_start} to {s0_end}", 
            f"{s1_name}: {s1_start} to {s1_end}",
            "Black indicates change outside the scale's range"
        ]
    },
).configure(
    font="Atkinson Hyperlegible"
).configure_title(
    fontSize=18,
    subtitleFontSize=14,
    anchor="middle"
).configure_view(
    strokeWidth=0
).configure_axisX(
    labelAngle=-30,
    labelFontSize=12
).project(
    "mercator"
)

alt.Chart(...)

In [11]:
alt.Chart(bgs).mark_geoshape().encode(
    color=alt.Color('supermarkets_t3_1-0:Q', title="Added Travel Time", scale=alt.Scale(domain=[-10,10]))
).properties(
    width=800,
    height=500,
    title=
).project(
    "mercator"
)

SyntaxError: invalid syntax (3114183552.py, line 7)

In [ ]:
compared.columns

Index(['bg_id', 'C000_c30_0', 'C000_c45_0', 'hospitals_t1_0',
       'rental_units_c30_0', 'rental_units_c45_0', 'corridor_acres_c30_0',
       'corridor_acres_c45_0', 'supermarkets_t1_0', 'supermarkets_t3_0',
       'C000_c30_1', 'C000_c45_1', 'hospitals_t1_1', 'rental_units_c30_1',
       'rental_units_c45_1', 'corridor_acres_c30_1', 'corridor_acres_c45_1',
       'supermarkets_t1_1', 'supermarkets_t3_1', 'C000_c30_1-0',
       'C000_c45_1-0', 'hospitals_t1_1-0', 'rental_units_c30_1-0',
       'rental_units_c45_1-0', 'corridor_acres_c30_1-0',
       'corridor_acres_c45_1-0', 'supermarkets_t1_1-0', 'supermarkets_t3_1-0'],
      dtype='object')

In [ ]:
demographics = pd.read_csv(f"../cache/{uid}/demographics.csv", dtype={"bg_id":str})
curve0 = compared[["bg_id", "C000_c45_0"]].copy()
curve0 = pd.merge(curve0, demographics, on="bg_id")
curve0 = curve0.sort_values("C000_c45_0")
curve0['f_everyone'] = curve0["B03002_001E"]/curve0["B03002_001E"].sum()
curve0["curve_total"] = curve0["f_everyone"].cumsum()
curve0["scenario"] = "Business as Usual"
curve0 = curve0.rename(columns={"C000_c45_0":"emp45"})

curve1 = compared[["bg_id", "C000_c45_1"]].copy()
curve1 = pd.merge(curve1, demographics, on="bg_id")
curve1 = curve1.sort_values("C000_c45_1")
curve1['f_everyone'] = curve1["B03002_001E"]/curve1["B03002_001E"].sum()
curve1["curve_total"] = curve1["f_everyone"].cumsum()
curve1["scenario"] = "Proposed Change"
curve1 = curve1.rename(columns={"C000_c45_1":"emp45"})

curves = pd.concat([curve0, curve1], axis="index")
curves

,bg_id,emp45,B03002_001E,B03002_003E,B03002_004E,B03002_006E,B03002_012E,C17002_003E,f_everyone,curve_total,scenario
686,420454081032,6.0,85,61,21,0,3,21,0.000031,0.000031,Business as Usual
1457,420912001062,24.0,712,495,22,109,35,0,0.000257,0.000288,Business as Usual
1903,420293054002,24.0,2406,1052,861,0,371,258,0.000870,0.001158,Business as Usual
1989,420454071022,33.0,869,790,14,34,11,17,0.000314,0.001472,Business as Usual
377,420293022043,39.0,2374,1769,16,328,188,0,0.000858,0.002330,Business as Usual
...,...,...,...,...,...,...,...,...,...,...,...
64,421019800032,559921.0,0,0,0,0,0,0,0.000000,0.999436,Proposed Change
282,421010004032,559991.0,965,758,27,63,78,31,0.000349,0.999785,Proposed Change
309,421010004031,562246.0,0,0,0,0,0,0,0.000000,0.999785,Proposed Change
941,421010005001,564836.0,596,247,173,70,33,91,0.000215,1.000000,Proposed Change


In [ ]:
alt.Chart(curves).mark_line().encode(
    alt.X("emp45:Q", title="Jobs Accessible in 45 Minutes"),
    alt.Y("curve_total:Q", scale=alt.Scale(domain=[0, 1]), title="Fraction of Population"),
    alt.Color("scenario:N", title="Scenario")
)

alt.Chart(...)